In [3]:
from urllib.request import urlopen
import json as json
import pandas as pd
import numpy as np
import requests

## ETL USA

In [2]:
url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&orderby=time&limit=10"
response= urlopen(url)
data= response.read()
print(data)

b'{"type":"FeatureCollection","metadata":{"generated":1683125256000,"url":"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&orderby=time&limit=10","title":"USGS Earthquakes","status":200,"api":"1.13.6","limit":10,"offset":1,"count":10},"features":[{"type":"Feature","properties":{"mag":0.32,"place":"9km WNW of Cobb, CA","time":1683124018780,"updated":1683124115414,"tz":null,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/nc73883456","detail":"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc73883456&format=geojson","felt":null,"cdi":null,"mmi":null,"alert":null,"status":"automatic","tsunami":0,"sig":2,"net":"nc","code":"73883456","ids":",nc73883456,","sources":",nc,","types":",nearby-cities,origin,phase-data,","nst":8,"dmin":0.008156,"rms":0.01,"gap":108,"magType":"md","type":"earthquake","title":"M 0.3 - 9km WNW of Cobb, CA"},"geometry":{"type":"Point","coordinates":[-122.8253326,38.8404999,2.22]},"id":"nc73883456"},\n{"type":"Feature","properties":{"m

In [156]:
url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&orderby=time"
df_usa= pd.read_csv(url)
pd.set_option('display.max_columns', None)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-05-09T19:49:03.864Z,63.429100,-147.664200,75.50,1.30,ml,NaN,NaN,NaN,0.62,ak,ak0235xnxilz,2023-05-09T19:51:13.034Z,"64 km E of Cantwell, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak
1,2023-05-09T19:38:29.230Z,33.867500,-117.507667,1.59,1.11,ml,16.0,65.0,0.07220,0.13,ci,ci40221799,2023-05-09T19:42:06.811Z,"2km SE of Home Gardens, CA",quarry blast,0.33,0.47,0.113,25.0,automatic,ci,ci
2,2023-05-09T19:36:10.050Z,38.839668,-122.835671,1.42,0.59,md,7.0,121.0,0.01654,0.01,nc,nc73885791,2023-05-09T19:59:12.404Z,"10km NW of The Geysers, CA",earthquake,0.41,1.01,0.430,8.0,automatic,nc,nc
3,2023-05-09T19:33:04.470Z,32.605500,-116.968500,3.02,1.45,ml,21.0,107.0,0.03241,0.27,ci,ci40221791,2023-05-09T19:36:38.784Z,"8km ESE of Chula Vista, CA",quarry blast,0.81,0.59,0.216,11.0,automatic,ci,ci
4,2023-05-09T19:25:28.760Z,38.841667,-122.811165,0.19,1.27,md,25.0,52.0,0.01609,0.03,nc,nc73885786,2023-05-09T19:54:10.374Z,"8km WNW of Cobb, CA",earthquake,0.16,0.41,0.160,26.0,automatic,nc,nc


In [157]:
df_usa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11465 entries, 0 to 11464
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             11465 non-null  object 
 1   latitude         11465 non-null  float64
 2   longitude        11465 non-null  float64
 3   depth            11465 non-null  float64
 4   mag              11465 non-null  float64
 5   magType          11465 non-null  object 
 6   nst              8146 non-null   float64
 7   gap              8146 non-null   float64
 8   dmin             6049 non-null   float64
 9   rms              11465 non-null  float64
 10  net              11465 non-null  object 
 11  id               11465 non-null  object 
 12  updated          11465 non-null  object 
 13  place            11161 non-null  object 
 14  type             11465 non-null  object 
 15  horizontalError  7511 non-null   float64
 16  depthError       11465 non-null  float64
 17  magError    

In [158]:
#elimino columnas innecesarias
df_usa= df_usa.drop(columns=['magType', 'nst', 'gap', 'dmin', 'rms', 'net', 'id','updated','type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status', 'locationSource', 'magSource'])
#coloco todos los registros en minusculas
df_usa= df_usa.applymap(lambda x: x.lower() if isinstance(x, str) else x)
#Vamos a redondear las columnas de variables float
df_usa[['latitude','longitude','depth','mag']]=df_usa[['latitude','longitude','depth','mag']].round(1)
#eliminamos duplicados
df_usa= df_usa.drop_duplicates()
#agrego una columna "country" con el nombre del pais respectivo en caso que necesite identificar en procesos posteriores
df_usa['country']='usa'

#cambiamos el formato de fecha para estandarizarla junto con los otros datasets
df_usa['time'] = pd.to_datetime(df_usa['time'])
df_usa['time'] = df_usa['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [173]:
df_usa

,time,latitude,longitude,depth,mag,place,country
0,2023-05-09 19:49:03,63.4,-147.7,75.5,1.3,"64 km e of cantwell, alaska",usa
1,2023-05-09 19:38:29,33.9,-117.5,1.6,1.1,"2km se of home gardens, ca",usa
2,2023-05-09 19:36:10,38.8,-122.8,1.4,0.6,"10km nw of the geysers, ca",usa
3,2023-05-09 19:33:04,32.6,-117.0,3.0,1.4,"8km ese of chula vista, ca",usa
4,2023-05-09 19:25:28,38.8,-122.8,0.2,1.3,"8km wnw of cobb, ca",usa
...,...,...,...,...,...,...,...
11460,2023-04-10 16:08:08,58.2,-155.3,1.0,0.3,"85 km nw of karluk, alaska",usa
11461,2023-04-10 16:06:40,51.9,-177.9,8.2,1.2,"88 km w of adak, alaska",usa
11462,2023-04-10 16:02:58,18.0,-67.0,14.2,2.3,"5 km wsw of fuig, puerto rico",usa
11463,2023-04-10 15:54:05,38.8,-122.7,1.6,1.2,"2km ne of the geysers, ca",usa


## ETL Japon

In [11]:
url = "https://www.jma.go.jp/bosai/quake/data/list.json"

response = requests.get(url)
data = response.json()

df_japon = pd.DataFrame(data)

In [12]:
df_japon.head()

,ctt,eid,rdt,ttl,ift,ser,at,anm,acd,cod,mag,maxi,int,json,en_ttl,en_anm
0,20230511013145,20230511010819,2023-05-11T01:31:00+09:00,遠地地震に関する情報,発表,1,2023-05-11T01:02:00+09:00,南太平洋,950,-15.6-174.6-210000/,7.6,,[],20230511013145_20230511010819_VXSE5e_1.json,Distant Earthquake Information,South Pacific Ocean
1,20230510215952,20230510215351,2023-05-10T21:59:00+09:00,震源・震度情報,発表,1,2023-05-10T21:53:00+09:00,岩手県沿岸北部,210,+39.8+141.9-60000/,3.7,1,"[{'code': '02', 'maxi': '1', 'city': [{'code':...",20230510215952_20230510215351_VXSE5k_1.json,Earthquake and Seismic Intensity Information,Northern Coast of Iwate Prefecture
2,20230510215815,20230510215452,2023-05-10T21:58:00+09:00,震源・震度情報,発表,1,2023-05-10T21:54:00+09:00,能登半島沖,495,+37.6+137.3-10000/,5.1,3,"[{'code': '17', 'maxi': '3', 'city': [{'code':...",20230510215815_20230510215452_VXSE5k_1.json,Earthquake and Seismic Intensity Information,Off the Coast of Noto Peninsula
3,20230510215732,20230510215452,2023-05-10T21:57:00+09:00,震源に関する情報,発表,0,2023-05-10T21:54:00+09:00,能登半島沖,495,+37.6+137.3-10000/,5.1,,[],20230510215732_20230510215452_VXSE52_0.json,Earthquake Information,Off the Coast of Noto Peninsula
4,20230510215622,20230510215452,2023-05-10T21:56:00+09:00,震度速報,発表,0,2023-05-10T21:54:00+09:00,,,,,3,"[{'code': '17', 'maxi': '3', 'city': []}]",20230510215622_20230510215452_VXSE51_0.json,Seismic Intensity Information,


In [13]:
#eliminamos las columnas que no vamos a utilizar
df_japon= df_japon.drop(columns=['ctt','eid','rdt','ttl','ift','ser','anm','acd','maxi','int','json','en_ttl'])
#eliminamos y/o reemplazamos caracateres innecesarios
df_japon['cod'] = df_japon['cod'].str.replace('+', '', 1) # Esta linea de codigo reemplaza exclusivamente el primer '+'
df_japon['cod'] = df_japon['cod'].str.replace('+', ',')
df_japon['cod'] = df_japon['cod'].str.replace('-', ',')
df_japon['cod'] = df_japon['cod'].str.replace('/', '')

#ahora que esta limpio podemos separar los datos
df_japon = df_japon.join(df_japon['cod'].str.split(',', expand=True).rename(columns={0:'latitude', 1:'longitude', 2:'depth'}))
df_japon = df_japon.drop(columns='cod')

#convertimos a float la columna "depth"
df_japon['depth'] = df_japon['depth'].astype('float64')
#dividimos por mil para llevar la unidad de medida a KM para mantener la misma en todos los datasets
df_japon['depth'] = (df_japon['depth'] / 1000)

#renombramos columnas
df_japon = df_japon.rename(columns={'at': 'time', 'en_anm': 'place'})
#reordenamos columnas
orden = ['time', 'latitude', 'longitude', 'depth', 'mag', 'place']
df_japon = df_japon[orden]

#todo a minusculas
df_japon=df_japon.applymap(lambda x: x.lower() if isinstance(x,str) else x)
#agrego una columna "country" con el nombre del pais respectivo en caso que necesite identificar en procesos posteriores
df_japon['country']='japon'

#reemplazamos los "ｍ不明" ("desconocido en español") de la columna "mag" por NaN
df_japon['mag'] = df_japon['mag'].replace({'ｍ不明': np.nan, '': np.nan})

#eliminamos duplicados
df_japon= df_japon.drop_duplicates()
#Vamos a redondear las columnas de variables float
df_japon[['latitude','longitude','depth','mag']]=df_japon[['latitude','longitude','depth','mag']].round(1)
#substituimos los Nones por NaN
df_japon['longitude'] = df_japon['longitude'].replace({None: np.nan, '': np.nan})
df_japon['latitude'] = df_japon['latitude'].replace({None: np.nan, '': np.nan})

#cambiamos el formato de fecha para estandarizarla junto con los otros datasets
df_japon['time'] = pd.to_datetime(df_japon['time'])
df_japon['time'] = df_japon['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

#elimino los registos nulos pero solo aquellos que son nulos en todas las columnas
df_lower = df_japon.dropna(subset=['latitude', 'longitude', 'depth', 'mag','place'], how='all')

/var/folders/_5/spzfhw090m5dhqvxrw9qgsbr0000gn/T/ipykernel_3026/2076281113.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_japon['cod'] = df_japon['cod'].str.replace('+', '', 1) # Esta linea de codigo reemplaza exclusivamente el primer '+'
/var/folders/_5/spzfhw090m5dhqvxrw9qgsbr0000gn/T/ipykernel_3026/2076281113.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_japon['cod'] = df_japon['cod'].str.replace('+', ',')


In [15]:
df_japon

,time,latitude,longitude,depth,mag,place,country
0,2023-05-11 01:02:00,NaN,15.6,0.2,7.6,south pacific ocean,japon
1,2023-05-10 21:53:00,39.8,141.9,60.0,3.7,northern coast of iwate prefecture,japon
2,2023-05-10 21:54:00,37.6,137.3,10.0,5.1,off the coast of noto peninsula,japon
4,2023-05-10 21:54:00,NaN,NaN,NaN,NaN,,japon
5,2023-05-10 17:29:00,37.4,137.3,10.0,3.0,toyama bay,japon
...,...,...,...,...,...,...,...
252,2023-04-12 00:44:00,27.0,141.6,0.0,4.2,adjacent sea of​ chichijima island,japon
253,2023-04-12 00:38:00,42.6,143.8,100.0,4.3,off the coast of tokachi,japon
254,2023-04-11 09:09:00,27.0,142.7,60.0,4.5,adjacent sea of​ chichijima island,japon
255,2023-04-10 22:10:00,56.7,161.4,NaN,NaN,adjacent kamchatka peninsula,japon


## ETL Chile

In [30]:
#traemos la informacion a traves de la API
url = "https://api.xor.cl/sismo/recent"

response = requests.get(url)
data = response.json()
df_chile = pd.DataFrame(data)
df_chile.head()

,status_code,status_description,events
0,0,Información obtenida satisfactoriamente,"{'id': '196628', 'url': 'http://sismologia.cl/..."
1,0,Información obtenida satisfactoriamente,"{'id': '196627', 'url': 'http://sismologia.cl/..."
2,0,Información obtenida satisfactoriamente,"{'id': '196612', 'url': 'http://sismologia.cl/..."
3,0,Información obtenida satisfactoriamente,"{'id': '196609', 'url': 'http://sismologia.cl/..."
4,0,Información obtenida satisfactoriamente,"{'id': '196606', 'url': 'http://sismologia.cl/..."


In [31]:
#Veamos que caracteristicas tienen las columnas
df_chile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   status_code         25 non-null     int64 
 1   status_description  25 non-null     object
 2   events              25 non-null     object
dtypes: int64(1), object(2)
memory usage: 728.0+ bytes


In [ ]:
#Elminamos las columnas innecesarias y nos quedamos solo con la columna "events"
df_chile= df_chile.drop(columns=['status_code','status_description'])

#Extraemos la informacion y la adjuntamos al dataframe como nuevas columnas
df_chile['time']= df_chile['events'].apply(lambda x : x['utc_date'])
df_chile['latitude']= df_chile['events'].apply(lambda x : x['latitude'])
df_chile['longitude']=df_chile['events'].apply(lambda x: x['longitude'])
df_chile['depth']=df_chile['events'].apply(lambda x: x['depth'])
df_chile['mag']=df_chile['events'].apply(lambda x: x['magnitude']['value'])
df_chile['place']=df_chile['events'].apply(lambda x: x['geo_reference'])

#Eliminamos la columna "events", ahora ya no la necesitamos
df_chile= df_chile.drop(columns=['events'])

#Pasamos todo a minusculas
df_chile= df_chile.applymap(lambda x : x.lower() if isinstance(x,str) else x)
#Agrego una columna "country" con el nombre del pais respectivo en caso que necesite identificar en procesos posteriores
df_chile['country']= 'chile'

#convertimos el tipo de la columna "depth" a float para mantener la misma estructura en todos los datasets
df_chile['depth']= df_chile['depth'].astype('Float64')
#eliminamos duplicados
df_chile= df_chile.drop_duplicates()
#Vamos a redondear las columnas de variables float
df_chile[['latitude','longitude','depth','mag']]=df_chile[['latitude','longitude','depth','mag']].round(1)

#cambiamos el formato de fecha para estandarizarla junto con los otros datasets
df_chile['time'] = pd.to_datetime(df_chile['time'])
df_chile['time'] = df_chile['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [177]:
df_chile

,time,latitude,longitude,depth,mag,place,country
0,2023-05-10 04:00:26,-21.1,-68.9,105.0,2.5,42 km al so de mina collahuasi,chile
1,2023-05-10 03:55:25,-31.8,-71.3,55.0,2.6,21 km al ne de los vilos,chile
2,2023-05-09 23:39:41,-26.1,-70.6,78.0,3.8,23 km al n de chañaral.,chile
3,2023-05-09 22:45:08,-21.2,-68.6,115.0,2.7,40 km al s de mina collahuasi,chile
4,2023-05-09 22:17:33,-24.8,-70.2,49.0,3.3,70 km al ne de taltal,chile
5,2023-05-09 20:45:31,-19.9,-69.2,100.0,3.0,67 km al n de pica,chile
6,2023-05-09 20:03:42,-36.1,-73.7,35.0,2.8,80 km al o de cobquecura,chile
7,2023-05-09 19:35:36,-36.1,-73.6,36.0,3.0,77 km al o de cobquecura,chile
8,2023-05-09 18:11:40,-21.7,-68.4,119.0,2.9,57 km al s de ollagüe,chile
9,2023-05-09 16:37:19,-40.4,-73.0,52.0,2.9,21 km al ne de osorno,chile


In [76]:
df_chile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       25 non-null     object 
 1   latitude   25 non-null     float64
 2   longitude  25 non-null     float64
 3   depth      25 non-null     int64  
 4   mag        25 non-null     float64
 5   place      25 non-null     object 
 6   country    25 non-null     object 
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ KB


In [179]:
df_final= pd.concat([df_usa,df_japon,df_chile])

In [180]:
df_final

,time,latitude,longitude,depth,mag,place,country
0,2023-05-09 19:49:03,63.4,-147.7,75.5,1.3,"64 km e of cantwell, alaska",usa
1,2023-05-09 19:38:29,33.9,-117.5,1.6,1.1,"2km se of home gardens, ca",usa
2,2023-05-09 19:36:10,38.8,-122.8,1.4,0.6,"10km nw of the geysers, ca",usa
3,2023-05-09 19:33:04,32.6,-117.0,3.0,1.4,"8km ese of chula vista, ca",usa
4,2023-05-09 19:25:28,38.8,-122.8,0.2,1.3,"8km wnw of cobb, ca",usa
...,...,...,...,...,...,...,...
20,2023-05-09 04:21:03,-21.2,-68.8,110.0,2.7,49 km al so de mina collahuasi,chile
21,2023-05-09 04:06:15,-23.4,-67.3,251.0,3.6,61 km al e de socaire,chile
22,2023-05-09 03:13:37,-21.1,-69.1,95.0,3.2,56 km al so de mina collahuasi,chile
23,2023-05-09 03:13:04,-35.0,-72.1,35.0,2.7,14 km al so de vichuquén,chile
